In [8]:
def filter(df):
    sos = signal.butter(N=3, Wn=30, btype='lowpass', fs=100, output='sos')
    df['Ax'] = signal.sosfilt(sos, df['Ax'])
    df['Ay'] = signal.sosfilt(sos, df['Ay'])
    df['Az'] = signal.sosfilt(sos, df['Az'])
    return df
    

In [9]:
def create_windows(df, time_steps, step):
    windows = []
    labels=[]
    for i in range(0, len(df)-time_steps, step):
        axs = df['Ax'].values[i: i + time_steps]
        ays = df['Ay'].values[i: i + time_steps]
        azs = df['Az'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        labels.append(stats.mode(df['label'][i: i + time_steps])[0][0])
        windows.append([axs, ays, azs])

    # Bring the segments into a better shape
    reshaped_windows = np.asarray(windows, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)
    return reshaped_windows, labels

In [7]:
def getStatisticalFeatures(data,name):
    columns=['mean'+name,'std'+name,'median'+name,'cov'+name,'twf_perc'+name,'svf_perc'+name,'maxi'+name,'mini'+name,'skewness'+name,'kurt'+name]
    res=pd.DataFrame(columns=columns)
    for i, window in enumerate(data):
        windowFlatten=window.flatten()
        maxi=np.amax(window)
        mini=np.amin(window)
        mean=np.mean(window, axis=None)
        std=np.std(window, axis=None)
        median=np.median(window, axis=None)
        cov=np.sqrt(std)
        twf_perc=np.percentile(window, 25, axis=None)
        svf_perc=np.percentile(window, 75, axis=None)
        skewness=skew(window)
        kurt=kurtosis(window)
        zero_crossings = len(np.where(np.diff(np.sign(windowFlatten)))[0])
        peak=len(signal.find_peaks(windowFlatten))
        rms=np.sqrt(np.mean(np.square(windowFlatten)))
        sampleEntropy=ant.sample_entropy(windowFlatten)
        spectralEntropy=ant.spectral_entropy(windowFlatten, sf=100, method='welch', normalize=True)
        crestFact=max(windowFlatten/rms)
        energy=np.sum(np.square(windowFlatten))
        res=res.append({'mean'+name:mean,
                    'std'+name:std,
                    'median'+name:median,
                    'cov'+name:cov,
                    'twf_perc'+name:twf_perc,
                    'svf_perc'+name:svf_perc,
                    'maxi'+name:maxi,
                    'mini'+name:mini,
                    'skewness'+name:skewness[0],
                    'kurt'+name:kurt[0],
                    'zeroCross'+name:zero_crossings,
                    'energy'+name: energy,
                    'peak'+name:peak,
                    'rms'+name: rms,
                    'sampleEntropy'+name:sampleEntropy,
                    'spectralEntropy'+name:spectralEntropy,
                    'crestFact'+name:crestFact},ignore_index=True)
    return res


In [11]:
def reshape_input(x,shape): #??????
    result=x.reshape(x.shape[0],shape)
    return result

In [8]:
def featureExtraction(df):
    dataWindowed,labels=create_windows(df, TIME_PERIODS,STEP_DISTANCE)
    statistics=[]
    statistics.append(getStatisticalFeatures(dataWindowed[:,:,0:1],'_Ax'))
    statistics.append(getStatisticalFeatures(dataWindowed[:,:,1:2],'_AY'))
    statistics.append(getStatisticalFeatures(dataWindowed[:,:,2:3],'_Az'))
    result=statistics[0]
    result=statistics[0].merge(statistics[1],right_index=True,left_index=True)
    result=result.merge(statistics[2],right_index=True,left_index=True)
    result['label']=labels
    result['ActivityEncoded'] = le.fit_transform(result['label'].values.ravel())
    return result

In [5]:
def getStatistics():
    files = sorted(glob.glob("C:\\Users\\emirc\\Desktop\\AAR\\codes\\statistics/*"))
    statistics={}
    for file in files:
        df=pd.read_csv(file)
        statistics[file.split('\\')[len(file.split('\\'))-1]]=df
        
    return statistics